# Setup

### Model decisions

In [1]:
# Weights and Biases
!pip install -q wandb
# Tensorflow
!pip install -q tensorflow


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
previous_loading = False
google_colab = False
moneyness = False
lags = 140
hyperparameter_search = False
training_size = 991232
random_seed = 0
finance_computers = False
fc_path = 'M:/Master/'
checkpoint_time = '11.05 1 mnd test sett full model run' # Insert checkpoint time if continuing training
starting_window = 0 # Insert starting window if continuing training
run_training = False
run_shap = True
run_prediction = False

In [3]:
if google_colab == True:
  from google.colab import drive
  drive.mount('/content/drive')

### Imports

In [4]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import MinMaxScaler
import wandb
from wandb.keras import WandbCallback
from datetime import datetime
from dateutil.relativedelta import relativedelta
from tensorflow import square, reduce_mean
from tensorflow.keras.losses import MSE
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
from tensorflow.math import multiply
from tensorflow.keras.metrics import MeanSquaredError, RootMeanSquaredError
from math import log
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker
from sklearn.metrics import mean_squared_error
import shap
from pathlib import Path

In [5]:
# If running in colab, insert your wandb key here

#import config
#Erlend
wandb.login(key='3cae81eb56be3190be5bb48c571e69933071df69')
# Hjalmar
#wandb.login(key="b47bcf387a0571c5520c58a13be35cda8ada0a99")


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

# Load, split and normalize data

### Load data

In [6]:
if google_colab:
    import tensorflow as tf
    # Print info
    gpu_info = !nvidia-smi
    gpu_info = '\n'.join(gpu_info)
    if gpu_info.find('failed') >= 0:
        print('Not connected to a GPU')
    else:
        print(gpu_info)
    
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

    # Code to read csv file into Colaboratory:
    !pip install -U -q PyDrive
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    from oauth2client.client import GoogleCredentials
    # Authenticate and create the PyDrive client.
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
    id = "1Doyuyo_VDOmJf0CLo5kl9XzMTfhGtxiR"
    downloaded = drive.CreateFile({'id':id}) 
    downloaded.GetContentFile('2010-2023_NSS_filtered_vF.csv')  
    df_read = pd.read_csv('2010-2023_NSS_filtered_vF.csv')
elif finance_computers:
    file = fc_path + "/data/processed_data/2010-2023_NSS_filtered_vF.csv"
    df_read = pd.read_csv(file)
else:
    file = "../data/processed_data/2010-2023_NSS_filtered_vF.csv"
    df_read = pd.read_csv(file)

display(df_read)

,Unnamed: 0,Quote_date,Price,Underlying_last,Strike,TTM,R
0,0,2010-01-04,207.490,1132.99,925.0,0.008219,0.00050
1,1,2010-01-04,182.500,1132.99,950.0,0.008219,0.00050
2,2,2010-01-04,157.500,1132.99,975.0,0.008219,0.00050
3,3,2010-01-04,132.600,1132.99,1000.0,0.008219,0.00050
4,4,2010-01-04,107.705,1132.99,1025.0,0.008219,0.00050
...,...,...,...,...,...,...,...
12057638,13739049,2023-03-31,217.750,4109.88,4700.0,1.726027,0.04198
12057639,13739050,2023-03-31,180.000,4109.88,4800.0,1.726027,0.04198
12057640,13739051,2023-03-31,146.550,4109.88,4900.0,1.726027,0.04198
12057641,13739052,2023-03-31,118.200,4109.88,5000.0,1.726027,0.04198


### Create lags

In [7]:
df = df_read
del df_read

if hyperparameter_search:
    df = df[df['Quote_date'] <= '2015-02-01']

if run_shap:
    df = df[df['Quote_date'] >= '2019-01-01']

# Group the data by Quote Date and calculate the mean for Underlying Price
df_agg = df.groupby('Quote_date').mean().reset_index()

# Values to returns
df_agg["Underlying_return"] = df_agg["Underlying_last"].pct_change()

# Add the Underlying Price Lag column
for i in range(1, lags + 1):
    df_agg['Underlying_' + str(i)] = df_agg['Underlying_return'].shift(i)

df = pd.merge(df, df_agg[['Quote_date', 'Underlying_return'] + ['Underlying_' + str(i) for i in range(1, lags + 1)]], on='Quote_date', how='left')
del df_agg

# Filter df between 2011-09-01
df = df[(df["Quote_date"] >= "2011-09-01")] if hyperparameter_search else df[(df["Quote_date"] >= "2011-12-01")]

print(df)

         Unnamed: 0  Quote_date     Price  Underlying_last  Strike       TTM  \
0           6063430  2019-01-02  1209.300          2509.98  1300.0  0.005479   
1           6063431  2019-01-02  1157.605          2509.98  1350.0  0.005479   
2           6063432  2019-01-02  1108.345          2509.98  1400.0  0.005479   
3           6063433  2019-01-02  1058.400          2509.98  1450.0  0.005479   
4           6063434  2019-01-02  1007.600          2509.98  1500.0  0.005479   
...             ...         ...       ...              ...     ...       ...   
6688863    13739049  2023-03-31   217.750          4109.88  4700.0  1.726027   
6688864    13739050  2023-03-31   180.000          4109.88  4800.0  1.726027   
6688865    13739051  2023-03-31   146.550          4109.88  4900.0  1.726027   
6688866    13739052  2023-03-31   118.200          4109.88  5000.0  1.726027   
6688867    13739053  2023-03-31    94.400          4109.88  5100.0  1.726027   

               R  Underlying_return  Un

### Format input data

In [8]:
# Format settings
max_timesteps = lags
bs_vars = ['Moneyness', 'TTM', 'R'] if moneyness else ['Underlying_last', 'Strike', 'TTM', 'R']
underlying_lags = [f'Underlying_{i}' for i in range (max_timesteps - 1, 0, -1)] + ['Underlying_return']

def create_rw_dataset(window_number = 0, df = None, shap = False):
    '''Creates dataset for a single rolling window period offsett by the window number'''

    # Create train, validation and test set split points
    test_months = 1
    train_start = datetime(2011,9,1) + relativedelta(months=window_number * test_months) if hyperparameter_search else datetime(2011,12,1) + relativedelta(months=window_number * test_months)
    val_start = train_start + relativedelta(months=3*12)
    test_start = val_start + relativedelta(months = 1 + test_months) if hyperparameter_search else val_start + relativedelta(months = 1)
    test_end = test_start + relativedelta(months=test_months)
    train_start = str(train_start.date())
    val_start = str(val_start.date())
    test_start = str(test_start.date())
    test_end = str(test_end.date())
        
    # Split train and validation data
    df_train = df[(df['Quote_date'] >= train_start) & (df['Quote_date'] < val_start)]
    df_val = df[(df['Quote_date'] >= val_start) & (df['Quote_date'] < test_start)]
    df_test = df[(df['Quote_date'] >= test_start) & (df['Quote_date'] < test_end)]

    del df

    # Extract target values
    train_y = (df_train['Price'] / df_train['Strike']).to_numpy() if moneyness else df_train['Price'].to_numpy()
    val_y = (df_val['Price'] / df_val['Strike']).to_numpy() if moneyness else df_val['Price'].to_numpy()
    test_y = (df_test['Price'] / df_test['Strike']).to_numpy() if moneyness else df_test['Price'].to_numpy()

    # If usining moneyness, extract strike
    if moneyness:
        train_strike = df_train['Strike'].to_numpy()
        val_strike = df_val['Strike'].to_numpy()
        test_strike = df_test['Strike'].to_numpy()


    # Print earliest and latest date in every dataframe used
    print("--------------Dataframe dates--------------")
    print(f"Train: {df_train['Quote_date'].min()} - {df_train['Quote_date'].max()}")
    print(f"Val: {df_val['Quote_date'].min()} - {df_val['Quote_date'].max()}")
    print(f"Test: {df_test['Quote_date'].min()} - {df_test['Quote_date'].max()}")
    print("-------------------------------------------")

    # Convert dataframes to numpy arrays
    train_x = [df_train[underlying_lags].to_numpy(), df_train[bs_vars].to_numpy()]
    val_x = [df_val[underlying_lags].to_numpy(), df_val[bs_vars].to_numpy()]
    test_x = [df_test[underlying_lags].to_numpy(), df_test[bs_vars].to_numpy()]

    del df_train
    del df_val

    # Scale features based on training set
    underlying_scaler = MinMaxScaler()
    train_x[0] = underlying_scaler.fit_transform(train_x[0].flatten().reshape(-1, 1)).reshape(train_x[0].shape)
    val_x[0] = underlying_scaler.transform(val_x[0].flatten().reshape(-1,1)).reshape(val_x[0].shape)
    test_x[0] = underlying_scaler.transform(test_x[0].flatten().reshape(-1,1)).reshape(test_x[0].shape)

    bs_scaler = MinMaxScaler()
    train_x[1] = bs_scaler.fit_transform(train_x[1])
    val_x[1] = bs_scaler.transform(val_x[1])
    test_x[1] = bs_scaler.transform(test_x[1])


    # Shuffle training set
    np.random.seed(random_seed)
    shuffle = np.random.permutation(len(train_x[0]))
    train_x = [train_x[0][shuffle], train_x[1][shuffle]]
    train_y = train_y[shuffle]

    # Extract training set
    train_x = [train_x[0][:training_size], train_x[1][:training_size]]
    train_y = train_y[:training_size]

    if moneyness:
        train_strike = train_strike[shuffle][:training_size]
    
    if shap:
        return train_x, test_x, train_start, val_start, test_start

    # Reshape data to fit LSTM
    train_x = [train_x[0].reshape(len(train_x[0]), max_timesteps, 1), train_x[1]]
    val_x = [val_x[0].reshape(len(val_x[0]), max_timesteps, 1), val_x[1]]
    test_x = [test_x[0].reshape(len(test_x[0]), max_timesteps, 1), test_x[1]]

    print(f'Train shape: {train_x[0].shape}, {train_x[1].shape}')
    print(f'Val shape: {val_x[0].shape}, {val_x[1].shape}')
    print(f'Test shape: {test_x[0].shape}, {test_x[1].shape}')

    if moneyness:
        return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike,
    return train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test

# Create the dataset for the first rolling window period
if not run_shap:
    if moneyness:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike = create_rw_dataset(df=df)
    else:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(df=df)
        if hyperparameter_search:
            del df

--------------Dataframe dates--------------
Train: nan - nan
Val: nan - nan
Test: nan - nan
-------------------------------------------


ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.

# Model construction

In [ ]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''

    # Input layers
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    # LSTM layers
    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.Interface_units,
        activation = tanh,
        return_sequences = False
    ))

    # MLP layers
    layers = Concatenate()([model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU(alpha=config.Leaky_relu_alpha)(layers)

    output = Dense(1, activation='relu')(layers)

    # Exponential decaying learning rate
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x[0])/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    # Compile model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    model.summary()
    return model

# Help functions

In [11]:
# Calculate the training and validation MSE loss on the actual option price when using price/strike as the target
def MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike):
    train_pred = model(train_x)
    val_pred = model(val_x)

    train_mse = reduce_mean(square((train_pred[:,0] - train_y)*train_strike))
    val_mse = reduce_mean(square((val_pred[:,0] - val_y)*val_strike))

    print(f' Training scaled MSE: {train_mse}, Validation scaled MSE: {val_mse}')

In [ ]:
import gc
from tensorflow.keras import backend as k

class ClearMemory(Callback):
    def on_epoch_end(self, epoch, logs=None):
        gc.collect()
        k.clear_session()

## Creating trainer function

In [9]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None, previous_checkpoint_path = None, checkpoint_path = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        # Build model and create callbacks
        if previous_checkpoint_path and os.path.exists(previous_checkpoint_path + ".h5"):
            model = load_model(previous_checkpoint_path + ".h5")
        else:
            model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )
        
        # Check if the checkpoint folder exists
        if checkpoint_path and not os.path.exists(checkpoint_path):
            # Create the checkpoint folder if it does not exist
            os.makedirs(checkpoint_path)
        
        if checkpoint_path:
          checkpoint = ModelCheckpoint(
              filepath=checkpoint_path + ".h5",
              monitor='val_loss',
              mode='min',
              save_best_only=True,
              save_weights_only=False
          )

        # Adapt sequence length to config
        train_x_adjusted = [train_x[0][:, :config.LSTM_timesteps, :], train_x[1]]
        val_x_adjusted = [val_x[0][:, :config.LSTM_timesteps, :], val_x[1]]
        print(f'Train shape: {train_x_adjusted[0].shape}, {train_x_adjusted[0].shape}')
        print(f'Val shape: {val_x_adjusted[0].shape}, {val_x_adjusted[0].shape}')

        # Train model
        model.fit(
            train_x_adjusted,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x_adjusted, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback, checkpoint, ClearMemory()] if checkpoint_path else [early_stopping, wandb_callback, ClearMemory()],
        )

        if moneyness:
            MSE_loss(model, train_x, train_y, train_strike, val_x, val_y, val_strike)

NameError: name 'train_x' is not defined

# Hyperparameter search

In [14]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v7.0',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [4, 8, 16, 32]},
        'Interface_units': {
            'values': [4, 8, 16, 32]},
        'MLP_units': {
            'values': [100, 200, 400, 600]},        
        'LSTM_timesteps': {
            'values': [20, 40, 80, 120]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 7, 'min': 2},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 7, 'min': 2},
        'Bn_momentum': {
            'values': [0.1, 0.4, 0.7, 0.99]},
        'Lr': {
            'distribution': 'log_uniform',
            'max': log(0.1), 'min': log(0.0002)},
        'Lr_decay': {
            'distribution': 'uniform',
            'max': 1, 'min': 0.8},
        'Leaky_relu_alpha': {
            'values': [0.01, 0.05, 0.1, 0.3]},
        'Minibatch_size': {
            'value': 4096},
        'Min_delta': {
            'value': 0.01 if moneyness else 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 3 if moneyness else 4},
        'Training_size': {
            'value': training_size},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
#sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing') 
sweep_id = 'kdb4nbpi'

if hyperparameter_search:
    wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing', count = 1000)

# Rolling window

In [10]:
def calculate_error(predictions, original):
    m = MeanSquaredError()
    m.update_state(predictions, original)
    print("MSE:", m.result().numpy())
    m = RootMeanSquaredError()
    m.update_state(predictions, original)
    print("RMSE:", m.result().numpy())

class config_object:
    def __init__(self, config):
        self.LSTM_units = config['LSTM_units']
        self.Interface_units = config['Interface_units']
        self.MLP_units = config['MLP_units']
        self.LSTM_timesteps = config['LSTM_timesteps']
        self.LSTM_layers = config['LSTM_layers']
        self.MLP_layers = config['MLP_layers']
        self.Bn_momentum = config['Bn_momentum']
        self.Lr = config['Lr']
        self.Lr_decay = config['Lr_decay']
        self.Minibatch_size = config['Minibatch_size']
        self.Min_delta = config['Min_delta']
        self.Patience = config['Patience']
        self.Num_features = config['Num_features']
        self.Architecture = config['Architecture']
        

In [19]:
num_windows = 99

config = {
    'LSTM_units': 8,
    'Interface_units': 4,
    'MLP_units': 200,
    'LSTM_timesteps': 140,
    'LSTM_layers': 3,
    'MLP_layers': 4,
    'Bn_momentum': 0.4,
    'Lr': 0.0564,
    'Lr_decay': 0.809,
    'Leaky_relu_alpha': 0.1,
    'Minibatch_size': 4096,
    'Min_delta': 0.01 if moneyness else 1,
    'Patience': 20,
    'Num_features': 3 if moneyness else 4, 
    'Architecture': 'LSTM-MLP v.E.2',
}

df_test_combined = pd.DataFrame()

if not checkpoint_time:
    checkpoint_time = datetime.now().strftime("%m-%d_%H-%M")
print(f'Checkpoint time: {checkpoint_time}')

previous_checkpoint_path = None

for window in range(starting_window, num_windows):
    np.random.seed(random_seed)
    tf.random.set_seed(random_seed)

    if moneyness:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test, train_strike, val_strike, test_strike = create_rw_dataset(df=df, window_number=window)
    else:
        train_x, train_y, val_x, val_y, test_x, test_y, train_start, val_start, test_start, df_test = create_rw_dataset(df=df, window_number=window)

    print('-------------------------------------------')
    print(f'Window {window + 1} of {num_windows}')
    print("Training start: ", train_start, "Validation start: ", val_start, "Test start: ", test_start)
    print("Loading previous checkpoint: ", previous_loading)
        
    if google_colab:
        checkpoint_path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/{checkpoint_time}/{train_start}/'
    elif finance_computers:
        checkpoint_path = f'{fc_path}/checkpoints/{checkpoint_time}/{train_start}/'
    else:
        checkpoint_path = f'./checkpoints/{checkpoint_time}/{train_start}/'

    config['Dataset'] = f'{train_start} - {val_start} - {test_start}'

    if run_training:
        trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = config, project = 'Deep learning for option pricing - rolling windows',  previous_checkpoint_path = previous_checkpoint_path, checkpoint_path = checkpoint_path)
    c_model = load_model(checkpoint_path + ".h5")
    
    # Split test_x into five parts
    part_size = len(test_x[0]) // 5
    test_x_parts = []
    for i in range(5):
        subset_1 = test_x[0][i * part_size:(i + 1) * part_size]
        subset_2 = test_x[1][i * part_size:(i + 1) * part_size]
        test_x_parts.append((subset_1, subset_2))

    # Adjust the last part to include any remaining data
    test_x_parts[-1] = (test_x[0][(len(test_x_parts) - 1) * part_size:], test_x[1][(len(test_x_parts) - 1) * part_size:])

    # Make predictions for each part
    predictions_parts = [np.array(c_model(part)) for part in test_x_parts]

    # Combine the predictions
    predictions = np.concatenate(predictions_parts, axis=0)

    print(f'--- Predictions for test_start {test_start} ---')
    calculate_error(predictions, test_y)
    print('-------------------------------------------')
    
    df_test["Prediction"] = predictions
    df_test_combined = pd.concat([df_test_combined, df_test[["Quote_date", "Price", "Prediction"] + bs_vars]])

    if previous_loading:
       previous_checkpoint_path = checkpoint_path
    
    del train_x
    del train_y
    del val_x
    del val_y
    del test_x
    del test_y
    del train_start
    del val_start
    del test_start
    del df_test

print(f"--- All model predictions ---")
calculate_error(df_test_combined["Prediction"], df_test_combined["Price"])
print("-------------------------------------------")

if google_colab == False:
    predictions_path = './predictions/'
    if checkpoint_path and not os.path.exists(predictions_path):
        os.makedirs(predictions_path)
    df_test_combined.to_csv(f'{predictions_path}{datetime.now().strftime("%m-%d_%H-%M")}.csv')
elif finance_computers:
    predictions_path = f'{fc_path}/predictions/'
    if checkpoint_path and not os.path.exists(predictions_path):
        os.makedirs(predictions_path)
    df_test_combined.to_csv(f'{predictions_path}{datetime.now().strftime("%m-%d_%H-%M")}.csv')
else:
  path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/06. Predictions/{checkpoint_time}.csv'
  df_test_combined.to_csv(path)

UnboundLocalError: local variable 'checkpoint_time' referenced before assignment

In [ ]:
# Define a function to calculate RMSE
def calculate_rmse(group):
    rmse_model_1 = np.sqrt(mean_squared_error(group["Price"], group["Prediction"]))
    return pd.Series({"RMSE_Model_1": rmse_model_1})

# Group data by Quote_date and calculate RMSE for each group
rmse_df = df_test_combined.groupby("Quote_date").apply(calculate_rmse).reset_index()

# Plot the RMSE values for each model
fig, ax = plt.subplots(figsize=(10, 6))
sns.lineplot(data=rmse_df, x="Quote_date", y="RMSE_Model_1", label="Model", ax=ax)

# Set the interval for x-axis labels
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=10))

plt.xlabel("Quote_date")
plt.ylabel("RMSE")
plt.title("RMSE for Models")
plt.legend()
plt.xticks(rotation=45)
plt.show()

# Shap values

In [11]:
c_model = None

def shapley_predict(x_2d):
    # Reshape data to fit LSTM
    x_3d = [x_2d[:, :lags].reshape(len(x_2d[:, :lags]), max_timesteps, 1), x_2d[:, lags:]]
    return np.array(c_model(x_3d))

def create_shapley_values(train_x_2d, test_x_2d):
    explainer = shap.KernelExplainer(shapley_predict, train_x_2d)
    shap_values = explainer.shap_values(test_x_2d)
    return explainer, shap_values

def shap_summary_all(shap_values, test_x_2d):
    shap.initjs()
    shap.summary_plot(shap_values[0], test_x_2d, feature_names = underlying_lags + bs_vars, max_display = 150)

def shap_summary_returns(shap_values, test_x_2d):
    shap.initjs()
    shap.summary_plot(shap_values[0][:, :lags], test_x_2d[:, :lags], feature_names = underlying_lags, max_display = 150)


In [14]:
kmeans = 75
sample_size = 5000
windows = 99
start_window = 99
window_interval = 1

df_shap_combined = pd.DataFrame()

model = None

timestamp = datetime.now()
timestamp = timestamp.strftime("%m-%d_%H-%M")

for window in range(start_window-1, windows, window_interval):
    # Load data
    train_x, test_x, train_x_org, train_start, val_start, test_start = create_rw_dataset(df=df, window_number=window, shap = True)

    if google_colab:
        checkpoint_path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/05. Checkpoints/{checkpoint_time}/{train_start}/'
    elif finance_computers:
        checkpoint_path = f'{fc_path}/checkpoints/{checkpoint_time}/{train_start}/'
    else:
        checkpoint_path = f'../checkpoints/{checkpoint_time}/{train_start}/'

    c_model = load_model(checkpoint_path + ".h5")

    train_x_2d_input = np.hstack((train_x[0], train_x[1]))
    test_x_2d_input = np.hstack((test_x[0], test_x[1]))
    train_x_2d_org_input = np.hstack((train_x_org[0], train_x_org[1]))

    train_x_2d = shap.kmeans(train_x_2d_input, kmeans)
    test_x_2d = shap.sample(train_x_2d_input, sample_size)
    test_x_2d = train_x_2d_input[:sample_size]
    test_x_2d_org = train_x_2d_org_input[:sample_size]

    explainer, shap_values = create_shapley_values(train_x_2d, test_x_2d)

    shap_summary_all(shap_values, test_x_2d)
    shap_summary_returns(shap_values, test_x_2d)

    df_shap = pd.DataFrame(shap_values[0], columns = underlying_lags + bs_vars)
    df_shap[underlying_lags + bs_vars] = test_x_2d

    if google_colab:
        save_path = f'/content/drive/MyDrive/01. Masters Thesis - Shared/06. Predictions/Shap/{timestamp}_Shap_{train_start}.csv'
    else:
        save_path = f'../results/Shap/{timestamp}_Shap_{train_start}.csv'
    df_shap.to_csv(save_path)

TypeError: weight_decay is not a valid argument, kwargs should be empty  for `optimizer_experimental.Optimizer`.